In [ ]:
import json
from itertools import product
import timeit
import pickle
from os import path, getenv
from dotenv import load_dotenv

load_dotenv()

In [ ]:
def make_prompt(context, command):
    framing = f'You are an AI that controls a smart home.'
    context = f'Here is the state of the devices in the home, in JSON format: {json.dumps(context)}'
    command = f'The user issues the command: {command}. Change the device state as appropriate.'
    formatting = f'Provide your response in JSON format.'

    return f'{framing} {context} {command} {formatting}'

def query_chatgpt(prompt):
    response = !chatgpt f'{prompt}'
    return response

In [ ]:
# contexts
contexts = {
    'simple': {
        'user': {
            'location': 'living_room'
        },
        'bedroom': {
            'lights': {
                'bedside_lamp': {
                    'state': 'off'
                }
            }
        },
        'living_room': {
            'lights': {
                'overhead': {
                    'state': 'off'
                },
                'lamp': {
                    'state': 'off'
                }
            }
        }
    },
    'medium': {
        'user': {
            'location': 'living_room'
        },
        'bedroom': {
            'lights': {
                'bedside_lamp': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                }
            }
        },
        'living_room': {
            'lights': {
                'overhead': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                },
               'lamp': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                }
            }
        }
    },
    'complex': {
        'user': {
            'location': 'living_room'
        },
        'bedroom': {
            'lights': {
                'bedside_lamp': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                }
            },
            'tvs': {
                'bedroom_tv': {
                    'state': 'off',
                    'volume': 20
                }
            }
        },
        'living_room': {
            'lights': {
                'overhead': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                },
                'lamp': {
                    'state': 'off',
                    'color': {
                        'r': 255,
                        'g': 255,
                        'b': 255
                    }
                }
            },
            'tvs': {
                'living_room_tv': {
                    'state': 'off',
                    'volume': 20
                }
            },
            'speakers': {
                'living_room_speaker': {
                    'state': 'off',
                    'volume': 20
                }
            }
        }
    }
}

In [ ]:
# commands
commands = {
    'simple': 'turn on the light',
    'medium': 'get ready for a party',
    'ambiguous': 'i am tired'
}

In [ ]:
trials = list(product(contexts.keys(), commands.keys()))

In [ ]:
def run_trial(trial):
    context = contexts[trial[0]]
    command = commands[trial[1]]
    prompt = make_prompt(context, command)

    t0 = timeit.default_timer()
    response = query_chatgpt(prompt)
    t1 = timeit.default_timer()

    with open(f'./results/pickles/{trial[0]}_{trial[1]}.pkl', 'ab') as f:
        pickle.dump(response, f)

    with open(f'./results/pickles/{trial[0]}_{trial[1]}_t.pkl', 'ab') as f:
        pickle.dump(t1 - t0, f)

In [ ]:
N = 10

for trial in trials:
    for _ in range(0, N):
        run_trial(trial)